# Image Viewer
Basic Camera Image Viewer Application

## Author
Jason Koglin,
Los Alamos National Laboratory

koglin@lanl.gov

## License
This analysis application is part of the catemis package, 
which is distributed as open-source software under a BSD 3-Clause License

© 2022. Triad National Security, LLC. All rights reserved.
LANL Copyright No. C21111.

This program was produced under U.S. Government contract 89233218CNA000001 for Los Alamos
National Laboratory (LANL), which is operated by Triad National Security, LLC for the U.S.
Department of Energy/National Nuclear Security Administration. All rights in the program are
reserved by Triad National Security, LLC, and the U.S. Department of Energy/National Nuclear
Security Administration. The Government is granted for itself and others acting on its behalf a
nonexclusive, paid-up, irrevocable worldwide license in this material to reproduce, prepare
derivative works, distribute copies to the public, perform publicly and display publicly, and to permit
others to do so.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors
   may be used to endorse or promote products derived from this software
   without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" 
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE 
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE 
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE 
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [ ]:
import matplotlib.pyplot as plt
SMALL_SIZE = 20
MEDIUM_SIZE = 25
BIGGER_SIZE = 30

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

fontsize = {
    'title': 20,
    'xlabel': 18,
    'yticks': 12,
    'ylabel': 18,
    'xticks': 12,
    'zlabel': 18,
    'cticks': 12,
}

from matplotlib.figure import Figure
def null_figure():
    return Figure(figsize=(8,6))

import numpy as np
import pandas as pd
import xarray as xr

import holoviews as hv
import hvplot.pandas
import hvplot.xarray

import panel as pn
import panel.widgets as pnw
import param

pn.extension()

In [ ]:
import platform
from pathlib import Path
import catemis

try:
    version = catemis.__version__

except:
    version = 'dev'
    
package = 'catemis'
package_git_url = 'https://git.lanl.gov/koglin/'+package

# needs update for documentation URL
docs_url = package_git_url

app_site = '{:} v{:}'.format(package, version)

package_path = Path(catemis.__file__).parent
lanl_logo_path = package_path/'template'/'LANL Logo White.png'
jdiv_logo_path = package_path/'template'/'j-div-logo-blue1.png'
if lanl_logo_path.is_file():
    lanl_logo_pn = pn.pane.PNG(str(lanl_logo_path), width=300)
else:
    lanl_logo_pn = None

if jdiv_logo_path.is_file():
    jdiv_logo_pn = pn.Row(
        pn.Spacer(width=50),
        pn.pane.PNG(str(jdiv_logo_path), width=200),
    )

else:
    jdiv_logo_pn = None

app_meta_pn = pn.Column(
    pn.pane.Markdown('###[App Documentation]({:})'.format(docs_url), 
                     align='center', width=320),
    pn.pane.Markdown('###Created by koglin@lanl.gov'),
    pn.pane.Markdown('###[{:}]({:}) v{:}'.format(package, package_git_url, version), 
                     align='center', width=320),
    align='center',
    width=320,
)

In [ ]:
def get_local_datetime(datetime_utc,
                       zone='us/mountain'):
    """
    Get the local datetime from datetime UTC
    (originally from darhtio.utils)

    Parameters
    ----------
    datetime : np.datetime64
        UTC datetime.
    zone : str, optional
        Time Zone. The default is 'us/mountain'.

    Returns
    -------
    np.datetime64
        Local datetime
    """
    import pandas as pd
    import pytz

    tz = pytz.timezone(zone)
    tstamp = pd.Timestamp(datetime_utc)
    datetime = (pd.Timedelta(tz.utcoffset(tstamp), units='ns')
                +datetime_utc).to_datetime64()

    return datetime

In [ ]:
status_pane = pn.pane.Markdown(object='', width=800)
plot_status_pane = pn.pane.Markdown(object='', width=400)
title_pane = pn.pane.Markdown(object='', width=400)
update_pane = pn.pane.Markdown(object='', width=200)

In [ ]:
path = str(Path(catemis.__file__).parent/'data')
data_folder = '20200701'
file_base = 'CAT_CAM.'

In [ ]:
class ImageViewer(param.Parameterized):
    action = param.Action(lambda x: x.param.trigger('action'), label='Update Plot')
    update_shots = param.Action(lambda x: x.param.trigger('update_shots'), label='Update Shots')
    height = param.Selector(default=1000, objects=[300,350,400,600,800,1000,1200,1600]) 
    width = param.Selector(default=1200, objects=[300,350,400,600,800,1000,1200,1600])
    pixel_size = param.Number(default=0.00375)
    xmin = param.Number(default=0, label='x min', step=5)
    xmax = param.Number(default=4.77, label='x max', step=5)
    ymin = param.Number(default=0, label='y min', step=5)
    ymax = param.Number(default=3.56, label='y max', step=5)
    x0 = param.Number(default=0, step=1, label='Xo (pixel)')
    y0 = param.Number(default=0, step=1, label='Yo (pixel)')
    cmin = param.Number(default=400, label='ADU max', step=5)
    cmax = param.Number(default=1200, label='ADU max', step=5)
    auto_lim = param.Boolean(default=True, label='auto limits')
    complete = param.Boolean(default=False, label='All Shots')
    auto = param.Boolean(default=True, label='Auto Param')
    rx = param.Number(default=0, step=0.25, label='Azimuth [deg]')
    ry = param.Number(default=0, step=0.25, label='Elevation [deg]')
    orientation = param.Number(default=0, step=0.25, label='Roll [deg]')
    use_xy_coords = param.Boolean(default=False)
    shots = param.Selector()
    files = param.Selector()
    use_shot_nums = param.Boolean(default=True)

    path = param.String(default=str(path))
    data_folder = param.String(default=data_folder)
    file_base = param.String(default=file_base, label='File Base')
    file_ext = param.String(default='.tif', label='Extension')

    def pn_select(self):
        if self.use_shot_nums:
            return pn.Row(self.param.shots, width=120)

        else:
            return pn.Row(self.param.files, width=320)
    
    @param.depends('update_shots', watch=True)
    def set_shots_from_path(self):
        data_path = Path(self.path)/self.data_folder
        files_path = list(data_path.glob('{:}*{:}'.format(self.file_base, self.file_ext)))
        files_list = [file.parts[-1] for file in files_path]
        self.param.files.objects = files_list
        
        shot_list = []
        print('setting shot list')
        for file in files_list:
            shot = file.split('.')[1]
            if shot.isnumeric():
                shot_list.append(int(shot))

        shot_list = sorted(shot_list)
        print('shots: ', shot_list)
        self.param.shots.objects = shot_list

        if self.use_shot_nums:
            if shot_list:
                self.shots = shot_list[0]
                self.param.trigger('action')

        else:
            if file_list:
                self.files = file_list[0]
                self.param.trigger('action')

    def load_image(self, **kwargs):
        """
        Read camera image using imageio (a default anaconda package)
        Simplified from moire_ana.get_image and darhtio.open_image methods.

        Returns
        -------
        xarray.DataArray

        """
        import os
        import imageio
        import xarray as xr
        from pathlib import Path
        
        shot = self.shots
        data_path = Path(self.path)/self.data_folder
        file_base = self.file_base
        file_ext = self.file_ext
        file_sel = self.files
        name = 'img'

        if self.use_shot_nums:
            # Select by shot number
            if not shot:
                print('ERROR: invalid shot')
                return

            fname = '{:}{:}{:}'.format(file_base, shot, file_ext)

        else:
            if not file_sel:
                print('ERROR: invalid file')
                return

            fname = file_sel
    
        print(fname)
        file_name = data_path/fname
        print(file_name)
    
        if not file_name.is_file():
            # Auto check for file type in case not formatted correctly
            print('... file not found {:}'.format(str(file_name)))
            print('WARNING:  File not formatted correctly ... trying to find correct file')
            try:
                flist = list(data_path.glob(file_base + '.' + str(shot))) \
                    + list(data_path.glob(fbase + str(shot) + '.*'))
            except:
                flist = None

            if not flist:
                print(
                    'ERROR:  File does not exist with base {:} and {:}'.format(
                        file_base, shot))
                return None

            file_name = str(flist[0])

        reader = imageio.get_reader(str(file_name))
        header = reader.get_meta_data()
        compression = header.pop('compression')

        data = reader.get_data(0)
        dims = ['y','x']

        da = xr.DataArray(data, dims=dims, name=name)
        da.coords['x'] = np.arange(da.shape[1])
        da.coords['x'].attrs['long_name'] = 'X'
        da.coords['y'] = np.arange(da.shape[0])
        da.coords['y'].attrs['long_name'] = 'Y'
        da.attrs.update(**header)
        da.attrs['path'] = str(data_path)
        da.attrs['file'] = file_base
        self.da = da

        return self.da
    
    @param.depends('action')
    def plot_image(self):
        shot = self.shots

        if self.use_shot_nums:
            title_pane.object = '# Select Shot'
            if not shot:
            # Select by shot number
                return

            title = 'Shot {:}'.format(shot)
            update_pane.object = '... Loading Shot {:}'.format(shot)
            
        else:
            title = self.files
            update_pane.object = '... Loading File {:}'.format(title)

        clabel = 'ADU'
        da = self.load_image()
        
        da.coords['X'] = self.pixel_size * (da.x - self.x0)
        da.coords['X'].attrs['long_name'] = 'X'
        da.coords['X'].attrs['units'] = 'mm'
        da.coords['Y'] = self.pixel_size * (da.y - self.y0)
        da.coords['Y'].attrs['long_name'] = 'Y'
        da.coords['Y'].attrs['units'] = 'mm'
        
        if self.use_xy_coords:
            xdim = 'X'
            ydim = 'Y'
            da = da.swap_dims({'x':'X', 'y':'Y'})

        else:
            xdim = 'x'
            ydim = 'y'

        if da is None:
            return null_figure()

        try:
            long_name = da.attrs.get('long_name', '')
            units = da.attrs.get('units', 'ADU')

            height = self.height
            width = self.width

            if self.auto_lim:
                try:
                    dfp = pd.DataFrame(da.values.flatten()).describe(percentiles=[0.01,0.99])
                    cmin = int(dfp.T['1%'])
                    cmax = int(dfp.T['99%'])
                    if False:
                        cmin -= 5
                        cmax += 5

                    self.cmin = cmin
                    self.cmax = cmax

                except:
                    print('Failed clim')
                    print(dfp)

            xlim = (self.xmin, self.xmax)
            ylim = (self.ymin, self.ymax)
            clim = (self.cmin, self.cmax)
            aspect = (self.ymax-self.ymin)/(self.xmax-self.xmin)

            plot_status_pane.object = '... Updating CatCam plot'

            plt_kwargs = {
                'fontsize': fontsize,
                'height': height,
                'width': width,
                'clim': clim,
                'clabel': clabel,
                'title': str(title),
            }
            if self.use_xy_coords:
                plt_kwargs['xlim'] = xlim
                plt_kwargs['ylim'] = ylim
        
            pn_plot = (
                da.hvplot.image(x=xdim, y=ydim, **plt_kwargs).opts(toolbar="above")
            )

        except:
            plot_status_pane.object = 'Failed Making Plot'
            pn_plot = null_figure()

        return pn_plot

    def cam_params(self):
        pn_params = pn.Column(
            pn.Row(
                pn.Column(
                    self.param.update_shots,
                    self.param.use_shot_nums,
                    width=160,
                ),
                pn.Column(
                    self.param.data_folder,
                    width=160,
                ),
                width=320,
            ),
            self.param.path,
            pn.Row(
                self.param.file_base,
                self.param.file_ext,
                width=320,
            ),
            pn.Row(
                self.pn_select, 
                width=320,
            ),
            pn.layout.Divider(),
            pn.Row(
                self.param.action,
                width=320,
            ),
            pn.Row(
                self.param.use_xy_coords,
                self.param.auto_lim, 
                width=213,
            ),
            pn.Row(
                self.param.pixel_size,
                self.param.cmin,
                self.param.cmax,
                width=320,
            ),
            pn.Row(
                self.param.x0,
                self.param.xmin,
                self.param.xmax,
                width=320,
            ),
            pn.Row(
                self.param.y0,
                self.param.ymin,
                self.param.ymax,
                width=320,
            ),
            pn.Row(
                self.param.width,
                self.param.height,
                width=320,
            ),
        )

        return pn_params
    
    def app(self):
        pn_app = pn.template.MaterialTemplate(
            title='Image Viewer', 
            logo=str(lanl_logo_path),
            header_background='#000F7E',
            theme=pn.template.DefaultTheme,
        )

        pn_app.sidebar.append(jdiv_logo_pn)
        pn_app.sidebar.append(self.cam_params)
        pn_app.sidebar.append(pn.layout.Divider())
        pn_app.sidebar.append(app_meta_pn)

        pn_app.main.append(
            pn.Column(
                pn.Card(self.plot_image, title='Image'),
            )
        )

        return pn_app    
    
viewer = ImageViewer()

In [ ]:
# To launch straight from Jupyter Notebook uncomment and execute this line
#viewer.app().show()

In [ ]:
# For use with command line as in documentation
# panel serve --show image_viewer.ipynb --port=5053
viewer.app().servable()